In [3]:
## Imports 

# General
import os
import pandas as pd
import random as rd
from tqdm.auto import tqdm
import spacy

# Local

# Redundency to avoid going higher than the repo level
while 'curate-108.ipynb' in os.listdir():
    os.chdir('..')
from FeatureExtractor import FeatureExtractor

In [4]:
## Load 108-goodbad.csv

# Go to the right directory

# Open the .csv
with open('data/108-goodbad.csv', encoding='utf-8') as file:
    dataframe = pd.read_csv(file)

# Display
dataframe.head(5)

,question,answer,quality
0,Quelles sont les régions les plus attractives ...,Les densités les plus faibles sont inférieures...,good
1,Quelle est la population de la France en 2019 ?,* La population augmente fortement en Afrique ...,bad
2,Quel est le rôle de l'État dans la réduction d...,L'organisation du territoire se caractérise ég...,bad
3,Quel est le rôle de l'État dans la réduction d...,L'organisation du territoire se caractérise do...,good
4,Quelles sont les inégalités de développement a...,* l'IDH des pays d'Afrique australe est encore...,good


In [5]:
## Formatting

# label column
mask = dataframe.loc[:, 'quality'] == 'good'
dataframe.loc[mask, 'label'] = 1
mask = dataframe.loc[:, 'quality'] != 'good'
dataframe.loc[mask, 'label'] = 0

# Drop useless columns
dataframe = dataframe.drop(columns=['answer', 'quality'])

# Display
dataframe.head(5)

,question,label
0,Quelles sont les régions les plus attractives ...,1.0
1,Quelle est la population de la France en 2019 ?,0.0
2,Quel est le rôle de l'État dans la réduction d...,0.0
3,Quel est le rôle de l'État dans la réduction d...,1.0
4,Quelles sont les inégalités de développement a...,1.0


In [6]:
## Visualization

for i, quality in zip(range(2), ('good', 'bad')):
    print(f"These questions have been classified as {quality}:")
    selected = list(dataframe.loc[dataframe.loc[:, 'label'] == i, 'question'])
    rd.shuffle(selected)
    for question in selected[:3]:
        print(f" - {question}")

These questions have been classified as good:
 - Comment est répartie la population en France ?
 - Pourquoi mettre en place une politique de développement durable en Afrique ?
 - Quelle est le rôle du souverain ?
These questions have been classified as bad:
 - Seuil
 - Suffrage égal
 - Quelles sont les causes du vieillissement de la population ?


In [7]:
## Prepare the features

# List of features
Features = list(FeatureExtractor('Ceci est une phrase de test.', restricted=True).extract_features(avoid=['_feature_8']).loc[:, 'name'])

# Changing some names
for i, feature in enumerate(Features):
    if feature.startswith('Moyenne de la fréquence des ') or feature.startswith('Ecart type de la fréquence des '):
        feature = feature.replace('Moyenne de la fréquence des ', 'M_')
        feature = feature.replace('Ecart type de la fréquence des ', 'SD_')
        feature = feature.replace(' dans le texte selon L3ST', '')
    Features[i] = feature

# Display
Features

C:\Users\meri2\.conda\envs\rorch\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\meri2\.conda\envs\rorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\meri2\.conda\envs\rorch\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\meri2\.conda\envs\rorch\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\meri2\.conda\envs\rorch\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['Verbes conjugués à la troisième personne du présent indicatif',
 'Déterminants possessifs',
 'Mots interrogatifs',
 'Nombre de mots',
 'Nombre de lettres',
 'Moyenne du nombre de lettres par mots',
 'Pronoms interogatifs',
 'Adverbes interogatifs',
 'Noms au pluriel',
 'Nombres cardinaux',
 'Verbes transitifs',
 'M_mots',
 'Interjections',
 'Lieux',
 'Lieux uniques',
 'Détails',
 'Détails uniques',
 'Organisations',
 'Organisations uniques',
 'Personnes',
 'Personnes uniques',
 'SD_mots',
 'M_adjectifs',
 'M_prépositions',
 'M_adverbes',
 'M_déterminants',
 'M_noms',
 'M_noms propres',
 'M_verbes',
 'SD_adjectifs',
 'SD_prépositions',
 'SD_adverbes',
 'SD_déterminants',
 'SD_noms',
 'SD_noms propres',
 'SD_verbes']

In [8]:
## Add the features

# Add the columns
dataframe.loc[:, Features] = 0

# NLP loading
nlp = spacy.load('fr_core_news_md')

# Loop for adding the values
for i, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]):
    dataframe.loc[i, Features] = list(FeatureExtractor(row['question'], nlp, restricted=True).extract_features(avoid=['_feature_8'], verbose=False).loc[:, 'value'])


# Display
dataframe.head(5)

100%|██████████| 2020/2020 [13:20<00:00,  2.52it/s]


,question,label,Verbes conjugués à la troisième personne du présent indicatif,Déterminants possessifs,Mots interrogatifs,Nombre de mots,Nombre de lettres,Moyenne du nombre de lettres par mots,Pronoms interogatifs,Adverbes interogatifs,...,M_noms,M_noms propres,M_verbes,SD_adjectifs,SD_prépositions,SD_adverbes,SD_déterminants,SD_noms,SD_noms propres,SD_verbes
0,Quelles sont les régions les plus attractives ...,1.0,0,0,0,10,48,4.8000,0,0,...,56.320000,0.0,NaN,255.915,0.00000,0.0,0.00,0.000000,0.0,NaN
1,Quelle est la population de la France en 2019 ?,0.0,0,0,0,10,38,3.8000,0,0,...,31.360000,0.0,NaN,0.000,9765.59000,NaN,0.00,0.000000,0.0,NaN
2,Quel est le rôle de l'État dans la réduction d...,0.0,0,0,1,16,69,4.3125,0,0,...,95.748000,NaN,NaN,0.000,10845.40707,NaN,0.00,109.447779,NaN,NaN
3,Quel est le rôle de l'État dans la réduction d...,1.0,0,0,1,16,69,4.3125,0,0,...,95.748000,NaN,NaN,0.000,10845.40707,NaN,0.00,109.447779,NaN,NaN
4,Quelles sont les inégalités de développement a...,1.0,0,0,0,13,65,5.0000,0,0,...,35.106667,0.0,NaN,256.170,12610.43000,NaN,6826.38,39.027656,0.0,NaN


In [9]:
## Saving the new version

dataframe.to_csv('data/curated-108.csv')